# Get previous Data
We will create examples for few-shot prompting and evaluation by parsing previous HN job postings. We will also build a list of requirement categorizations that we can expect.

In [ ]:
#| default_exp get_examples

Create several example outputs by hand.

In [ ]:
examples = """
Example 1.
    Posting: 

        Rubie | Founding engineer | New York City | Full time | Onsite\nRubie is helping companies with complex software (ERPs, CRMs, etc) launch with their customers faster. We do this by automating the highly repetitive manual tasks that professional services and customer success teams perform using novel agentic infrastructure. This results in a massively faster customer implementation cycle - in the case of one of our partners, 70% faster!\nWe are hiring founding engineers in person in NYC. The first step in our process is completing our reverse engineering challenge, which is located here: https://ReverseEngineerThisSite.com\nreply

    Output:

    ```json
    [{
        "company": "Rubie",
        "job_title": "Founding engineer",
        "location":  "New York City",
        "work_schedule": "Full time",
        "remote/local/hybrid details": "Local only",
        "company_goal": "helping companies with complex software (ERPs, CRMs, etc) launch with their customers faster by automating the highly repetitive manual tasks that professional services and customer success teams perform using novel agentic infrastructure.",
        "job_requirements": "Experience with complex software (ERPs, CRMs, etc), ability to complete reverse engineering challenge",
        "contact_url": "https://ReverseEngineerThisSite.com",
        "application_process": "The first step in our process is completing our reverse engineering challenge, which is located here: https://ReverseEngineerThisSite.com",
    }]
    ```

Example 2.
    Posting: 

        MongoDB-- we're looking for an experienced Java developer who has expertise in building applications with Hibernate ORM (even better if you are familiar with hibernate-ogm) to help build a Hibernate integration with MongoDB. High priority + impact + high visibility role!\nFully REMOTE (global) | Software Engineer, Java (Database Experience Team) | Full-Time | Base + RSU's | Interested? More info + apply here: https://grnh.se/e29caf031us\nreply

    Output:

    ```json
    [{
        "company": "MongoDB",
        "job_title": "Software Engineer, Java (Database Experience Team)",
        "job_description": "High priority + impact + high visibility role!",
        "remote/local details": "Fully REMOTE(global)",
        "job_requirements": "experienced Java developer who has expertise in building applications with Hibernate ORM (even better if you are familiar with hibernate-ogm)",
        "contact_url": "https://grnh.se/e29caf031us",
        "tech_stack": ["Java", "Hibernate ORM"],
        "compensation": "Base + RSU's",
    }]
    ```

Example 3.

    Posting: 

        VALIS Insights | Full-time | REMOTE (US)\nI'm the co-founder and CTO of VALIS, a B2B SaaS company delivering software solutions to the metals recycling industry. Our goal is to reduce global carbon emissions through maximizing the recovery of metals through recycling, which requires a fraction of the energy compared to mining and primary metals production. We're a seed-stage startup, backed by leading climate-tech VC's.\nProduct-wise, we integrate recycling processes, equipment/sensors, production, and procurement/sales data to deliver an insights platform that helps metals recovery facilities optimize their processes. We employ a strong, domain-driven design approach to building our product. We're looking for execution-focused engineers to join our team and deliver complex features with autonomy.\nWe're currently hiring for two roles:\nSenior Backend Engineer - https://apply.workable.com/valis-insights-1/j/4745C9DC32/\nFull-Stack Engineer - https://apply.workable.com/valis-insights-1/j/3A06E4C40A/\nPlease reach out to me at caleb.ralphs@valisinsights.com and apply if you're interested.\nreply

    Output:
    
    ```json

    [{
        "company": "VALIS Insights",
        "job_title": "Senior Backend Engineer",
        "job_requirements": "Execution-focused engineers who can deliver complex features with autonomy",
        "contact_url": "caleb.ralphs@valisinsights.com",
        "work_schedule": "Full-time",
        "remote/local details": "REMOTE (US)",
        "company_goal": "reduce global carbon emissions through maximizing the recovery of metals through recycling, which requires a fraction of the energy compared to mining and primary metals production.",
        "company_stage": "seed",
        "urls": ["https://apply.workable.com/valis-insights-1/j/4745C9DC32/", "https://apply.workable.com/valis-insights-1/j/3A06E4C40A/"],
        "additional_notes": ["backed by leading climate-tech VC's.\nProduct-wise, we integrate recycling processes, equipment/sensors, production, and procurement/sales data to deliver an insights platform that helps metals recovery facilities optimize their processes.","We employ a strong, domain-driven design approach to building our product."]
    }
    {
        "company": "VALIS Insights",
        "job_title": "Full-Stack Engineer",
        "job_requirements": "Execution-focused engineers who can deliver complex features with autonomy",
        "contact_url": "caleb.ralphs@valisinsights.com",
        "work_schedule": "Full-time",
        "remote/local details": "REMOTE (US)",
        "company_goal": "reduce global carbon emissions through maximizing the recovery of metals through recycling, which requires a fraction of the energy compared to mining and primary metals production.",
        "company_stage": "seed",
        "urls": ["https://apply.workable.com/valis-insights-1/j/4745C9DC32/", "https://apply.workable.com/valis-insights-1/j/3A06E4C40A/"],
        "additional_notes": ["backed by leading climate-tech VC's. Product-wise, we integrate recycling processes, equipment/sensors, production, and procurement/sales data to deliver an insights platform that helps metals recovery facilities optimize their processes.","We employ a strong, domain-driven design approach to building our product."],
    }]
    ```

Example 4.
    Posting: 

        Duolingo | Multiple Roles | Hybrid | Pittsburgh/Seattle/NYC | Full-time | $148,800-$274,600 + equity/benefits\nHere at Duolingo, we are passionate about educating our users, making fact-based decisions, and finding innovative solutions to complex problems. We offer meaningful work, limitless learning opportunities, and collaboration with world-class minds. Come brighten your life and over half a billion more!\nHighlighted Roles:\nDatabase Reliability Engineer (PGH)\nPlatform Engineer (PGH)\nSenior Mobile Engineers, iOS and Android (PGH, NYC, Seattle)\nSenior Backend Engineers (PGH)\nEngineering Leadership, Manager or Director (PGH)\nSenior/Staff Data Scientists, Economics (PGH or NYC)\nFor more details, you can view/apply to all current openings here: https://grnh.se/41f1af102us\n            ,     ,\n            )\\___/(\n           {(@)v(@)}\n            {|~~~|}\n            {/^^^\\}\n             `m-m`\nreply

    Output:
    ```json
    [{
        "company": "Duolingo",
        "job_title": "Multiple Roles",
        "job_requirements": "Available on the website link provided",
        "contact_url": "https://grnh.se/41f1af102us",
        "remote/local details": "Hybrid",
        "compensation": "$148,800-$274,600 + equity/benefits",
        "company_goal": "we are passionate about educating our users, making fact-based decisions, and finding innovative solutions to complex problems.",
        "additional_notes": "We offer meaningful work, limitless learning opportunities, and collaboration with world-class minds. Come brighten your life and over half a billion more!"
    },
    {
        "company": "Duolingo",
        "job_title": "Database Reliability Engineer",
        "job_requirements": "Available on the website link provided",
        "contact_url": "https://grnh.se/41f1af102us",
        "remote/local details": "Hybrid",
        "location": "Pittsburgh (PGH)",
        "compensation": "$148,800-$274,600 + equity/benefits",
        "company_goal": "we are passionate about educating our users, making fact-based decisions, and finding innovative solutions to complex problems.",
        "additional_notes": "We offer meaningful work, limitless learning opportunities, and collaboration with world-class minds. Come brighten your life and over half a billion more!"
    }]
    ```
"""

In [ ]:
from langchain_core.prompts import PromptTemplate

getPostsPrompt = PromptTemplate.from_template("""You will be returning one ore more json objects together in an array to describe the jobs in a job posting. Here are some examples:\n\n{examples}\n\n Here are the key values currently being used to categorize jobs:\n\n{key_values}.\n\n If there is only one job posting presented here, return a json object containing a summary of every separate piece of information in the job posting. If the job describes multiple roles, create a separate json object for each role. Use the existing key values where they are relevant. If there is no relevant existing key, add new, relevant key to your json object. Here is the job posting: \n\n{job_posting}""")



1. Get last month's  job listings

In [ ]:
import json

# Opening JSON file
f = open('../posts.json')

# returns JSON object as 
# a dictionary
posts = json.load(f)

2. Create key/value pairs from each post
* update the keys as we go

The LLM returns the data surrounded by comments and block markers. We need to strip them to get the actual json.

In [ ]:
#| export

def extract_middle_json(response):
    # Split the string on '```'
    parts = response.split('```')
    # Select the middle portion and remove 'json' from the front
    return parts[1].replace('json', '').strip() if len(parts) > 1 else ''

def extractJSON(response):
    jsonStr = extract_middle_json(response)

    return json.loads(jsonStr)
    

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

parsed_data = [  ]
result = {}

# Start with some keys we are sure will be needed
keys = [
    'job_title',
    'company',
    'location',
    'employment_type',
    'tech_stack',
    'job_requirements',
    'salary',
]

postItems = posts

for i in range(len(postItems)):
        msg = getPostsPrompt.format(examples = examples, job_posting = postItems[i]['comment'], key_values = ", ".join(keys))
        resp = model.invoke(msg)
        parsed_datum = extractJSON(resp.content)
        
        for  job in  parsed_datum:
            parsed_data.append(job)

            parsed_data_keys = list(job.keys()) 
    
            for key in parsed_data_keys:
                if key not in keys:
                    keys.append(key)

            # We add keys here, so subsequent examples can use them
            keyStr = ", ".join(keys)

        print(parsed_data)


In [ ]:
resp.content

In [ ]:

# let's see how frequently each key appears
tally = {}
for key in keys:
    tally[key] = sum([1 for datum in parsed_data if key in datum])

tally

Save the keys for use, and human editing


In [ ]:
from hn_jobs_chat.util import saveJSON

saveJSON('keys.json', keys)
saveJSON('parsed_data-july.json', parsed_data)

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()